In [1]:
!pip install pandas numpy srt regex tqdm spacy rank-bm25 faiss-cpu chromadb openai
!python -m spacy download en_core_web_sm

  Using cached pandas-2.3.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached numpy-2.3.4-cp313-cp313-macosx_14_0_arm64.whl.metadata (62 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached faiss_cpu-1.12.0-cp313-cp313-macosx_14_0_arm64.whl.metadata (5.1 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached typing_inspection-0.4.2-py3-none-any.whl.metadata (2.6 kB)
  Using cache

In [4]:
!pip install python-dotenv kaggle

  Using cached webencodings-0.5.1-py2.py3-none-any.whl.metadata (2.1 kB)
Using cached webencodings-0.5.1-py2.py3-none-any.whl (11 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [kaggle]2m4/5 [kaggle]


In [1]:
# # Get Open AI key from .env
# import os
# os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here"

import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


In [6]:
# Get from Kaggle - https://www.kaggle.com/datasets/gunnvant/game-of-thrones-srt
from dotenv import load_dotenv
import os, json
load_dotenv()  # call early

kaggle_api = os.getenv("KAGGLE_API_KEY")
if kaggle_api:
    try:
        creds = json.loads(kaggle_api)
        os.environ["KAGGLE_USERNAME"] = creds.get("username", os.getenv("KAGGLE_USERNAME"))
        os.environ["KAGGLE_KEY"] = creds.get("key", os.getenv("KAGGLE_KEY"))
    except Exception:
        # fall back to separate vars if JSON parse fails
        pass

# Ensure both vars exist
if not (os.getenv("KAGGLE_USERNAME") and os.getenv("KAGGLE_KEY")):
    raise EnvironmentError("Set KAGGLE_USERNAME and KAGGLE_KEY in .env")

import json
import pandas as pd
from kaggle.api.kaggle_api_extended import KaggleApi

# Initialize and authenticate the Kaggle API
api = KaggleApi()
try:
    api.authenticate()
except Exception as e:
    raise RuntimeError(f"Failed to authenticate Kaggle API: {e}")

# Define the Kaggle dataset path
dataset_path = "gunnvant/game-of-thrones-srt"  # Replace if needed

# Download and unzip the dataset to the ./data directory
os.makedirs("./data", exist_ok=True)
print(f"Downloading dataset '{dataset_path}' from Kaggle to ./data ...")
api.dataset_download_files(dataset_path, path="./data", unzip=True)
print("Dataset downloaded and unzipped successfully.")

Dataset URL: https://www.kaggle.com/datasets/gunnvant/game-of-thrones-srt
Dataset downloaded and unzipped successfully.


In [10]:
# ingest_kaggle_json.py
from pathlib import Path
import json, regex as re
import pandas as pd

RAW_JSON_DIR = Path("data")  # put season1.json ... season7.json here

# Extract S/E from "Game Of Thrones S01E01 Winter Is Coming.srt"
SE_RE = re.compile(r"[Ss](\d{1,2})[Ee](\d{1,2})")

def parse_episode_key(ep_key: str):
    m = SE_RE.search(ep_key)
    season = int(m.group(1)) if m else None
    episode = int(m.group(2)) if m else None
    return season, episode

def load_kaggle_season_json(p: Path, seconds_per_line: float = 2.5) -> list[dict]:
    with open(p, "r", encoding="utf-8") as f:
        data = json.load(f)  # dict: episode_filename -> { "1": "line", "2": "line", ... }

    rows = []
    for ep_key, lines_obj in data.items():
        season, episode = parse_episode_key(ep_key)
        # lines_obj keys are strings of integers; sort numerically
        line_items = sorted(lines_obj.items(), key=lambda kv: int(kv[0]) if kv[0].isdigit() else kv[0])
        for idx, (_, text) in enumerate(line_items):
            if not text or not str(text).strip():
                continue
            # synthesize simple timestamps so downstream chunkers work
            t_start = idx * seconds_per_line
            t_end   = t_start + seconds_per_line
            rows.append({
                "season": season,
                "episode": episode,
                "t_start": float(t_start),
                "t_end": float(t_end),
                "text": str(text).strip()
            })
    return rows

# ---- Ingest all season*.json files ----
all_rows = []
for p in sorted(RAW_JSON_DIR.glob("season*.json")):
    all_rows.extend(load_kaggle_season_json(p, seconds_per_line=2.5))

df_lines = pd.DataFrame(all_rows).dropna(subset=["text"])
# keep S1–S7 if desired
df_lines = df_lines[(df_lines["season"] >= 1) & (df_lines["season"] <= 7)]
df_lines = df_lines.sort_values(["season", "episode", "t_start"]).reset_index(drop=True)

print("Loaded lines:", len(df_lines))
print(df_lines.head())

Loaded lines: 44890
   season  episode  t_start  t_end  \
0       1        1      0.0    2.5   
1       1        1      2.5    5.0   
2       1        1      5.0    7.5   
3       1        1      7.5   10.0   
4       1        1     10.0   12.5   

                                                text  
0                                         Easy, boy.  
1               What do you expect? They're savages.  
2            One lot steals a goat from another lot,  
3  before you know it they're ripping each other ...  
4    I've never seen wildlings do a thing like this.  


In [19]:
# ...existing code...
# Single guarded retrieval setup (embeddings, FAISS, BM25, hybrid_search)
if not globals().get("_GOT_RETRIEVAL_SETUP_DONE", False) or globals().get("_bm25_row_count", None) != len(globals().get("df_chunks", [])):
    print("Running one-time retrieval setup: embeddings/FAISS/BM25 (or rebuilding because df changed).")

    import numpy as np
    from pathlib import Path
    from rank_bm25 import BM25Okapi
    from tqdm import tqdm

    # embed model (single definition)
    EMBED_MODEL = "text-embedding-3-large"

    # safe embedder
    def embed_texts(texts, batch_size=16, max_chars=8000):
        import time
        vecs = []
        for i in tqdm(range(0, len(texts), batch_size), desc="Embedding"):
            batch = texts[i:i+batch_size]
            batch_trunc = [t if len(t) <= max_chars else t[:max_chars] + " [TRUNCATED]" for t in batch]
            try:
                resp = client.embeddings.create(model=EMBED_MODEL, input=batch_trunc)
                vecs.extend([e.embedding for e in resp.data])
            except Exception as e:
                print(f"Batch embed failed at idx {i}: {e}. Falling back to per-item embedding.")
                for t in batch_trunc:
                    for attempt in range(3):
                        try:
                            r = client.embeddings.create(model=EMBED_MODEL, input=[t])
                            vecs.extend([e.embedding for e in r.data])
                            break
                        except Exception as e2:
                            print(f"  item embed attempt {attempt+1} failed: {e2}")
                            time.sleep(1 + attempt * 2)
                    else:
                        raise RuntimeError("Embedding failed repeatedly for an item. Consider reducing max_chars.")
        return np.asarray(vecs, dtype="float32")

    # file candidates (prefer existing artifacts)
    vec_paths = [Path("data/got_vecs.npy"), Path("data/got_aug_vecs.npy"), Path("data/got_chunks_vecs.npy")]
    index_paths = [Path("data/got_faiss.bin"), Path("data/got_aug_faiss.bin"), Path("data/got_chunks_faiss.bin")]
    df_paths = [Path("data/got_chunks.csv"), Path("data/got_aug_chunks.csv")]

    # ensure df_chunks exists or load snapshot
    if "df_chunks" not in globals() or df_chunks is None:
        loaded = False
        for p in df_paths:
            if p.exists():
                df_chunks = pd.read_csv(p)
                print(f"Loaded chunk dataframe from {p}")
                loaded = True
                break
        if not loaded:
            raise FileNotFoundError("df_chunks not in memory and no snapshot found at data/got_chunks.csv or data/got_aug_chunks.csv. Run chunking step first.")

    # build / load vectors + FAISS index (prefer existing artifacts)
    vec_path = next((p for p in vec_paths if p.exists()), None)
    index_path = next((p for p in index_paths if p.exists()), None)

    if vec_path and index_path:
        vecs = np.load(vec_path)
        try:
            index = faiss.read_index(str(index_path))
        except Exception as e:
            print(f"Failed to read FAISS index at {index_path}: {e}. Rebuilding index from vectors.")
            faiss.normalize_L2(vecs)
            index = faiss.IndexFlatIP(vecs.shape[1])
            index.add(vecs)
            faiss.write_index(index, str(index_path))
            print(f"Rebuilt and saved FAISS index → {index_path}")
        print(f"Loaded vectors ({vec_path}) and FAISS index ({index_path}) — ntotal={index.ntotal}")
    else:
        print("No precomputed vectors/index found — computing embeddings and building FAISS index.")
        vecs = embed_texts(df_chunks["text"].fillna("").tolist())
        faiss.normalize_L2(vecs)
        index = faiss.IndexFlatIP(vecs.shape[1])
        index.add(vecs)
        out_vec_path = Path("data/got_aug_vecs.npy")
        out_index_path = Path("data/got_aug_faiss.bin")
        np.save(out_vec_path, vecs)
        faiss.write_index(index, str(out_index_path))
        print(f"Saved vectors → {out_vec_path}  and FAISS index → {out_index_path}")

    print("FAISS index ready:", index.ntotal, "vectors")

    # build BM25 (guarded)
    corpus_tokens = [[tok.text.lower() for tok in nlp(txt)] for txt in tqdm(df_chunks["text"].fillna(""), desc="Tokenizing")]
    # Initialize BM25
    print("Building BM25 index...")
    bm25 = BM25Okapi(corpus_tokens)
    _bm25_row_count = len(df_chunks)

    # mark setup done
    _GOT_RETRIEVAL_SETUP_DONE = True
else:
    print("Reusing existing retrieval artifacts (BM25 + FAISS already built).")

# single hybrid_search definition (use df_chunks)
# ...existing code...
def hybrid_search(query: str, topk: int = 10, alpha: float = 0.35, cand_mult: int = 20):
    import numpy as np
    # defensive check: index may have different ntotal than df_chunks
    if index.ntotal != len(df_chunks):
        print(f"Warning: FAISS index size ({index.ntotal}) != df_chunks rows ({len(df_chunks)}). "
              "Filtering out-of-range vector hits.")

    q_emb = client.embeddings.create(model=EMBED_MODEL, input=[query]).data[0].embedding
    qv = np.asarray(q_emb, dtype="float32")[None, :]
    faiss.normalize_L2(qv)
    D, I = index.search(qv, topk * cand_mult)
    vec_scores, vec_idx = D[0].tolist(), I[0].tolist()

    # filter FAISS results to valid dataframe indices
    max_valid = len(df_chunks)
    valid_vec_pairs = [(int(idx), float(score)) for idx, score in zip(vec_idx, vec_scores)
                       if isinstance(idx, (int, np.integer)) and 0 <= int(idx) < max_valid]
    v_map = {i: s for i, s in valid_vec_pairs}
    vec_idx_valid = [i for i, _ in valid_vec_pairs]

    q_tokens = [t.text.lower() for t in nlp(query)]
    bm_scores = bm25.get_scores(q_tokens)
    bm_top = np.argsort(bm_scores)[::-1][:topk * cand_mult]

    # ensure BM25 indices are valid (should be, but be defensive)
    bm_top_valid = [int(i) for i in bm_top if 0 <= int(i) < max_valid]

    cand = list(set(vec_idx_valid) | set(bm_top_valid))
    if not cand:
        import pandas as pd
        return pd.DataFrame([])

    bm_max = max(bm_scores) if len(bm_scores) else 1.0

    scored = []
    for i in cand:
        v = float(v_map.get(int(i), 0.0))
        b = float(bm_scores[int(i)]) / (bm_max + 1e-6)
        scored.append((int(i), alpha * v + (1 - alpha) * b))
    scored.sort(key=lambda x: x[1], reverse=True)

    rows = []
    for i, sc in scored[:topk]:
        # double-check index bounds before accessing df
        if 0 <= int(i) < len(df_chunks):
            r = df_chunks.iloc[int(i)].to_dict()
            r["score"] = float(sc)
            rows.append(r)
    import pandas as pd
    return pd.DataFrame(rows)
# ...existing code...

# quick smoke test (optional)
if globals().get("_GOT_RETRIEVAL_SETUP_DONE", False):
    try:
        res = hybrid_search("Who kills Tywin Lannister?", topk=5)
        print(res[["season", "episode", "score"]])
        print("Top text sample:\n", res.iloc[0]["text"][:300])
    except Exception as _e:
        print("Hybrid search smoke test skipped/failed:", _e)
# ...existing code...

Reusing existing retrieval artifacts (BM25 + FAISS already built).
   season  episode     score
0       4        6  0.832710
1       5        8  0.675566
2       1        3  0.641983
3       1        8  0.602664
4       4        5  0.589426
Top text sample:
 - Ser Davos... - Humour me. - Tywin Lannister. - How old is Tywin Lannister? Sixty-seven. And when he dies, who's in command? A half-grown boy, the product of incest?


In [16]:
!pip install pyarrow

  Using cached pyarrow-22.0.0-cp313-cp313-macosx_12_0_arm64.whl.metadata (3.2 kB)
Using cached pyarrow-22.0.0-cp313-cp313-macosx_12_0_arm64.whl (34.2 MB)


In [21]:
import pandas as pd
from pathlib import Path

GOLD = Path("eval/golden_set.csv")
dfg = pd.read_csv(GOLD)

def contains_all(text: str, must_tokens: list[str]) -> bool:
    t = text.lower()
    return all(tok.lower() in t for tok in must_tokens)

def recall_at_k_eval(df_gold, k=10):
    hits = 0
    total = 0
    rows = []
    for r in df_gold.itertuples():
        q = r.question_text
        ents = [] if pd.isna(r.entities) else [t.strip() for t in str(r.entities).split(";") if t.strip()]
        res = hybrid_search(q, topk=k, alpha=0.5)
        joined = " ".join(res["text"].tolist())
        ok = contains_all(joined, ents) if ents else (len(res) > 0)
        hits += 1 if ok else 0
        total += 1
        rows.append({"question_id": r.question_id, "ok": ok, "retrieved": len(res)})
    print(f"Retrieval Recall@{k}: {hits}/{total} = {hits/total:.2f}")
    return pd.DataFrame(rows)

eval_rows = recall_at_k_eval(dfg, k=10)
print(eval_rows.head())

Retrieval Recall@10: 2/3 = 0.67
  question_id     ok  retrieved
0          q1   True         10
1          q2  False         10
2          q3   True         10


In [28]:

import pandas as pd
from pathlib import Path

GOLD = Path("eval/golden_set.csv")
if not GOLD.exists():
    raise FileNotFoundError(f"{GOLD} not found. Place golden_set.csv at {GOLD}")

dfg = pd.read_csv(GOLD)

def contains_all(text: str, must_tokens: list[str]) -> bool:
    t = text.lower()
    return all(tok.lower() in t for tok in must_tokens)

def recall_at_k_eval(df_gold, k=10):
    hits, total, rows = 0, 0, []
    for r in df_gold.itertuples():
        q = r.question_text
        ents = [] if pd.isna(r.entities) else [t.strip() for t in str(r.entities).split(";") if t.strip()]
        # ensure search_fn is defined (hybrid_search/search_fn cell must run first)
        res = search_fn(q, topk=k)
        joined = " ".join(res["text"].tolist()) if len(res) else ""
        ok = contains_all(joined, ents) if ents else (len(res) > 0)
        hits += int(ok); total += 1
        rows.append({"question_id": r.question_id, "ok": ok, "retrieved": len(res)})
    out = pd.DataFrame(rows)
    print(f"Retrieval Recall@{k}: {hits}/{total} = {hits/total:.2f}  |  Rerank={'on' if USE_RERANK else 'off'}")
    return out

eval_rows = recall_at_k_eval(dfg, k=10)
eval_rows.head()

import pandas as pd
from pathlib import Path

GOLD = Path("eval/golden_set.csv")
if not GOLD.exists():
    raise FileNotFoundError(f"{GOLD} not found. Place golden_set.csv at {GOLD}")

dfg = pd.read_csv(GOLD)

def contains_all(text: str, must_tokens: list[str]) -> bool:
    t = text.lower()
    return all(tok.lower() in t for tok in must_tokens)

def recall_at_k_eval(df_gold, k=10):
    hits, total, rows = 0, 0, []
    for r in df_gold.itertuples():
        q = r.question_text
        ents = [] if pd.isna(r.entities) else [t.strip() for t in str(r.entities).split(";") if t.strip()]
        # ensure search_fn is defined (hybrid_search/search_fn cell must run first)
        res = search_fn(q, topk=k)
        joined = " ".join(res["text"].tolist()) if len(res) else ""
        ok = contains_all(joined, ents) if ents else (len(res) > 0)
        hits += int(ok); total += 1
        rows.append({"question_id": r.question_id, "ok": ok, "retrieved": len(res)})
    out = pd.DataFrame(rows)
    print(f"Retrieval Recall@{k}: {hits}/{total} = {hits/total:.2f}  |  Rerank={'on' if USE_RERANK else 'off'}")
    return out

eval_rows = recall_at_k_eval(dfg, k=10)
eval_rows.head()
# ...existing code...

Retrieval Recall@10: 2/3 = 0.67  |  Rerank=off
Retrieval Recall@10: 2/3 = 0.67  |  Rerank=off
Retrieval Recall@10: 2/3 = 0.67  |  Rerank=off
Retrieval Recall@10: 2/3 = 0.67  |  Rerank=off


,question_id,ok,retrieved
0,q1,True,10
1,q2,False,10
2,q3,True,10


In [22]:
# ==========================
# Phase 3-A (fixed) — Minimal grounded answerer
# ==========================
from openai import OpenAI
import os, pandas as pd
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

SEARCH_FN = hybrid_search  # or hybrid_search_reranked
GEN_MODEL = "gpt-4o-mini"

ANSWER_PROMPT = """You must answer ONLY using the EVIDENCE below.
If the evidence is insufficient, say: "I can’t answer confidently from the provided episodes."
Return a short answer, then list citations as S{{season}}E{{episode}} {{start}}-{{end}}s.

Question: {question}

EVIDENCE
{evidence}
"""

def format_evidence_rows(df_hits: pd.DataFrame, max_chars: int = 11000) -> str:
    def safe_get(r, key, default=""):
        return r[key] if key in r and pd.notna(r[key]) else default
    def fmt(r):
        s = safe_get(r, "season", "?")
        e = safe_get(r, "episode", "?")
        t1 = safe_get(r, "t_start", 0.0)
        t2 = safe_get(r, "t_end", 0.0)
        text = safe_get(r, "text", "")
        return f"S{s}E{e} {t1:.0f}-{t2:.0f}s :: {text}"
    lines, total = [], 0
    for _, r in df_hits.iterrows():
        s = fmt(r)
        if total + len(s) > max_chars:
            break
        lines.append(s)
        total += len(s)
    return "\n".join(lines)

def generate_answer(question: str, topk: int = 6, search_fn=SEARCH_FN):
    hits = search_fn(question, topk=topk)
    # Ensure we always have season/episode columns
    for col in ["season", "episode", "t_start", "t_end", "text"]:
        if col not in hits.columns:
            hits[col] = None
    ev_text = format_evidence_rows(hits)
    prompt = ANSWER_PROMPT.format(question=question, evidence=ev_text)
    out = client.chat.completions.create(
        model=GEN_MODEL,
        temperature=0.2,
        messages=[{"role":"user","content": prompt}]
    )
    answer = out.choices[0].message.content.strip()
    return answer, hits

# Smoke test
ans, hits = generate_answer("Who kills Tywin Lannister?")
print(ans)

Tyrion Lannister kills Tywin Lannister with a bolt to the heart.

Citations: S5E8 22-38s


In [35]:
!pip install -q requests beautifulsoup4 lxml

In [41]:
# Jupyter cell: robust FunTrivia GOT crawler — extracts qnum, question, short answer, explanation, source_url

import requests, time, urllib.parse
from bs4 import BeautifulSoup
from pathlib import Path
import pandas as pd
from tqdm import tqdm

HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; funtrivia-scraper/1.0; +https://github.com/you)",
    "Accept-Language": "en-US,en;q=0.9",
}
BASE = "https://www.funtrivia.com"
START = "https://www.funtrivia.com/en/Television/Game-of-Thrones-20275.html"

def parse_questions_from_soup(soup, source_url):
    out = []
    for q_div in soup.find_all(attrs={"itemtype":"http://schema.org/Question"}):
        # number
        step = q_div.find(class_="step")
        qnum = step.get_text(strip=True) if step else None

        # question text (itemprop=name)
        qname = q_div.find(attrs={"itemprop":"name"})
        question = qname.get_text(" ", strip=True) if qname else None

        # accepted answer block
        ans_bq = q_div.find("blockquote", class_="answer")
        short_ans = ""
        explanation = ""
        if ans_bq:
            # find all itemprop="text" elements inside blockquote (first often contains short answer, second explanation)
            texts = ans_bq.find_all(attrs={"itemprop":"text"})
            if texts:
                # Prefer bold inside first text for concise answer
                first = texts[0]
                b = first.find("b")
                if b and b.get_text(strip=True):
                    short_ans = b.get_text(strip=True)
                else:
                    # fallback: strip "Answer:" prefix if present
                    t0 = first.get_text(" ", strip=True)
                    short_ans = t0.replace("Answer:", "").strip()
                if len(texts) > 1:
                    explanation = texts[1].get_text(" ", strip=True)
                else:
                    # sometimes explanation is directly after the bold inside same node
                    # try to remove the bold text from first node and use remaining as explanation
                    if b:
                        # remove bold tag content to get any trailing explanation text
                        for tag in first.find_all("b"):
                            tag.decompose()
                        rem = first.get_text(" ", strip=True)
                        if rem:
                            explanation = rem
            else:
                # last-resort: take blockquote text and try to split first sentence as short answer
                full = ans_bq.get_text(" ", strip=True)
                if ":" in full:
                    # common "Answer: X Explanation..."
                    parts = full.split(":", 1)
                    short_ans = parts[1].split()[0]
                    explanation = parts[1].strip()
                else:
                    short_ans = full
        out.append({
            "qnum": qnum,
            "question": question,
            "answer_short": short_ans,
            "explanation": explanation,
            "source_url": source_url
        })
    return out

def crawl_funtrivia(start_url=START, max_q=200, max_pages=50, pause=0.9):
    seen_q = set()
    results = []
    url = start_url
    pages = 0
    while url and pages < max_pages and len(results) < max_q:
        pages += 1
        try:
            resp = requests.get(url, headers=HEADERS, timeout=20)
            resp.raise_for_status()
        except Exception as e:
            print(f"failed {url}: {e}")
            break
        soup = BeautifulSoup(resp.text, "lxml")
        parsed = parse_questions_from_soup(soup, url)
        for item in parsed:
            key = (item["qnum"], item["question"])
            if key in seen_q:
                continue
            seen_q.add(key)
            results.append(item)
            if len(results) >= max_q:
                break

        # find next page (link rel="next" or pager anchors)
        next_link = None
        link_tag = soup.find("link", rel="next")
        if link_tag and link_tag.get("href"):
            next_link = urllib.parse.urljoin(BASE, link_tag["href"])
        else:
            # fallback: look for anchor with pattern _2.html etc.
            pager = soup.select_one(".pagelist, .pagelinks, .pages")
            if pager:
                a_next = pager.find("a", string=lambda s: s and s.strip().isdigit() and int(s.strip()) == pages+1)
                if a_next and a_next.get("href"):
                    next_link = urllib.parse.urljoin(BASE, a_next["href"])
            # last fallback: look for any "Next" text
            if not next_link:
                a_next2 = soup.find("a", string=lambda s: s and "next" in s.lower())
                if a_next2 and a_next2.get("href"):
                    next_link = urllib.parse.urljoin(BASE, a_next2["href"])

        url = next_link
        time.sleep(pause)

    df = pd.DataFrame(results)
    return df

# run crawler (adjust max_q as needed)
df_all = crawl_funtrivia(max_q=200, max_pages=40, pause=0.8)

outdir = Path("data")
outdir.mkdir(parents=True, exist_ok=True)
outpath = outdir / "funtrivia_questions_all_200.csv"
df_all.to_csv(outpath, index=False)
print(f"Scraped {len(df_all)} QA pairs -> {outpath}")
df_all.head()
#```# filepath: /Users/carlosrortiz/Documents/csc5830-ThroneRag/got_eda.ipynb
# Jupyter cell: robust FunTrivia GOT crawler — extracts qnum, question, short answer, explanation, source_url
!pip install -q requests beautifulsoup4 lxml tqdm

import requests, time, urllib.parse
from bs4 import BeautifulSoup
from pathlib import Path
import pandas as pd
from tqdm import tqdm

HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; funtrivia-scraper/1.0; +https://github.com/you)",
    "Accept-Language": "en-US,en;q=0.9",
}
BASE = "https://www.funtrivia.com"
START = "https://www.funtrivia.com/en/Television/Game-of-Thrones-20275.html"

def parse_questions_from_soup(soup, source_url):
    out = []
    for q_div in soup.find_all(attrs={"itemtype":"http://schema.org/Question"}):
        # number
        step = q_div.find(class_="step")
        qnum = step.get_text(strip=True) if step else None

        # question text (itemprop=name)
        qname = q_div.find(attrs={"itemprop":"name"})
        question = qname.get_text(" ", strip=True) if qname else None

        # accepted answer block
        ans_bq = q_div.find("blockquote", class_="answer")
        short_ans = ""
        explanation = ""
        if ans_bq:
            # find all itemprop="text" elements inside blockquote (first often contains short answer, second explanation)
            texts = ans_bq.find_all(attrs={"itemprop":"text"})
            if texts:
                # Prefer bold inside first text for concise answer
                first = texts[0]
                b = first.find("b")
                if b and b.get_text(strip=True):
                    short_ans = b.get_text(strip=True)
                else:
                    # fallback: strip "Answer:" prefix if present
                    t0 = first.get_text(" ", strip=True)
                    short_ans = t0.replace("Answer:", "").strip()
                if len(texts) > 1:
                    explanation = texts[1].get_text(" ", strip=True)
                else:
                    # sometimes explanation is directly after the bold inside same node
                    # try to remove the bold text from first node and use remaining as explanation
                    if b:
                        # remove bold tag content to get any trailing explanation text
                        for tag in first.find_all("b"):
                            tag.decompose()
                        rem = first.get_text(" ", strip=True)
                        if rem:
                            explanation = rem
            else:
                # last-resort: take blockquote text and try to split first sentence as short answer
                full = ans_bq.get_text(" ", strip=True)
                if ":" in full:
                    # common "Answer: X Explanation..."
                    parts = full.split(":", 1)
                    short_ans = parts[1].split()[0]
                    explanation = parts[1].strip()
                else:
                    short_ans = full
        out.append({
            "qnum": qnum,
            "question": question,
            "answer_short": short_ans,
            "explanation": explanation,
            "source_url": source_url
        })
    return out

def crawl_funtrivia(start_url=START, max_q=200, max_pages=50, pause=0.9):
    seen_q = set()
    results = []
    url = start_url
    pages = 0
    while url and pages < max_pages and len(results) < max_q:
        pages += 1
        try:
            resp = requests.get(url, headers=HEADERS, timeout=20)
            resp.raise_for_status()
        except Exception as e:
            print(f"failed {url}: {e}")
            break
        soup = BeautifulSoup(resp.text, "lxml")
        parsed = parse_questions_from_soup(soup, url)
        for item in parsed:
            key = (item["qnum"], item["question"])
            if key in seen_q:
                continue
            seen_q.add(key)
            results.append(item)
            if len(results) >= max_q:
                break

        # find next page (link rel="next" or pager anchors)
        next_link = None
        link_tag = soup.find("link", rel="next")
        if link_tag and link_tag.get("href"):
            next_link = urllib.parse.urljoin(BASE, link_tag["href"])
        else:
            # fallback: look for anchor with pattern _2.html etc.
            pager = soup.select_one(".pagelist, .pagelinks, .pages")
            if pager:
                a_next = pager.find("a", string=lambda s: s and s.strip().isdigit() and int(s.strip()) == pages+1)
                if a_next and a_next.get("href"):
                    next_link = urllib.parse.urljoin(BASE, a_next["href"])
            # last fallback: look for any "Next" text
            if not next_link:
                a_next2 = soup.find("a", string=lambda s: s and "next" in s.lower())
                if a_next2 and a_next2.get("href"):
                    next_link = urllib.parse.urljoin(BASE, a_next2["href"])

        url = next_link
        time.sleep(pause)

    df = pd.DataFrame(results)
    return df

# run crawler (adjust max_q as needed)
df_all = crawl_funtrivia(max_q=200, max_pages=40, pause=0.8)

outdir = Path("data")
outdir.mkdir(parents=True, exist_ok=True)
outpath = outdir / "funtrivia_questions_all_200.csv"
df_all.to_csv(outpath, index=False)
print(f"Scraped {len(df_all)} QA pairs -> {outpath}")
df_all.head()

Scraped 165 QA pairs -> data/funtrivia_questions_all_200.csv
Scraped 165 QA pairs -> data/funtrivia_questions_all_200.csv


,qnum,question,answer_short,explanation,source_url
0,1,Who are the two brothers of the Night's Watch ...,Brant and Derek,Sam is beaten half to death trying to defend G...,https://www.funtrivia.com/en/Television/Game-o...
1,2,"Daenerys is traded away as a bride to a ""savag...",Viserys,Prince Viserys (Harry Lloyd) is the older and ...,https://www.funtrivia.com/en/Television/Game-o...
2,3,The Night King has so far been the main villai...,Viserion,The Night King kills Viserion with a fantastic...,https://www.funtrivia.com/en/Television/Game-o...
3,4,"At the conclusion of season 7, Cersei Lanniste...",The Golden Company,The Golden Company is an an elite group of sel...,https://www.funtrivia.com/en/Television/Game-o...
4,5,What is the name of Jon's direwolf?,Ghost,"Ghost, unlike the other direwolves found by th...",https://www.funtrivia.com/en/Television/Game-o...


In [45]:
# language: python
# Cell: test/eval_first_20_funtrivia (to place into test_got_eda.ipynb)
from pathlib import Path
import pandas as pd

# Try to import functions from got_eda (absolute import as requested)
try:
    from got_eda import crawl_funtrivia, parse_questions_from_soup
except Exception as _e:
    # Import may fail in notebook context; keep names for diagnostics
    crawl_funtrivia = None
    parse_questions_from_soup = None

# Accept both possible filenames the notebook used
candidates = [
    Path("eval/funtrivia_questions_all_200.csv")
]

csv_path = next((p for p in candidates if p.exists()), None)
if csv_path is None:
    raise FileNotFoundError(
        "funtrivia CSV not found. Run the crawler cell first to produce data/funtrivia_questions_all_200.csv"
    )

df = pd.read_csv(csv_path)
n_check = 20
df_head = df.head(n_check).copy()

required_cols = {"qnum", "question", "answer_short", "explanation", "source_url"}

missing_cols = required_cols - set(df_head.columns)
if missing_cols:
    raise AssertionError(f"Missing expected columns in CSV: {missing_cols}")

# Tests / checks
failures = []
passed = 0
for idx, row in df_head.iterrows():
    row_id = f"row_index={idx}"
    q = str(row.get("question", "")).strip() if pd.notna(row.get("question", "")) else ""
    ans_short = str(row.get("answer_short", "")).strip() if pd.notna(row.get("answer_short", "")) else ""
    expl = str(row.get("explanation", "")).strip() if pd.notna(row.get("explanation", "")) else ""
    src = str(row.get("source_url", "")).strip() if pd.notna(row.get("source_url", "")) else ""

    row_fail_reasons = []
    if not q:
        row_fail_reasons.append("empty_question")
    if not (ans_short or expl):
        row_fail_reasons.append("no_answer_or_explanation")
    if src and not (src.startswith("http://") or src.startswith("https://")):
        row_fail_reasons.append("bad_source_url")

    if row_fail_reasons:
        failures.append({"index": idx, "reasons": row_fail_reasons, "question": q})
    else:
        passed += 1

# Summary and assertions so cell fails on test failures
print(f"CSV checked: {csv_path}  |  total_rows_in_file={len(df)}  |  checking_first={n_check}")
print(f"Passed: {passed}/{n_check}  |  Failures: {len(failures)}")
if failures:
    print("Failures detail (showing up to 10):")
    for f in failures[:10]:
        print(f"- index {f['index']}: reasons={f['reasons']}  | question_preview={f['question'][:120]!r}")

# Final assertion to make this behave like a test: require all 20 to pass
assert len(failures) == 0, f"{len(failures)} of first {n_check} rows failed basic validation. See printed failures."

# Optional: if crawler function imported, show indication it's available
if crawl_funtrivia is not None:
    print("Imported crawl_funtrivia from got_eda (available for further tests).")
else:
    print("crawl_funtrivia not imported (not available via absolute import).")

# Display the first 5 rows for manual inspection
display(df_head.head(5))

CSV checked: eval/funtrivia_questions_all_200.csv  |  total_rows_in_file=165  |  checking_first=20
Passed: 20/20  |  Failures: 0
crawl_funtrivia not imported (not available via absolute import).


,qnum,question,answer_short,explanation,source_url
0,1,Who are the two brothers of the Night's Watch ...,Brant and Derek,Sam is beaten half to death trying to defend G...,https://www.funtrivia.com/en/Television/Game-o...
1,2,"Daenerys is traded away as a bride to a ""savag...",Viserys,Prince Viserys (Harry Lloyd) is the older and ...,https://www.funtrivia.com/en/Television/Game-o...
2,3,The Night King has so far been the main villai...,Viserion,The Night King kills Viserion with a fantastic...,https://www.funtrivia.com/en/Television/Game-o...
3,4,"At the conclusion of season 7, Cersei Lanniste...",The Golden Company,The Golden Company is an an elite group of sel...,https://www.funtrivia.com/en/Television/Game-o...
4,5,What is the name of Jon's direwolf?,Ghost,"Ghost, unlike the other direwolves found by th...",https://www.funtrivia.com/en/Television/Game-o...


In [23]:
# ...existing code...
import pandas as pd
from pathlib import Path

# Accept the new filename (primary) and fallbacks
candidates = [
    Path("eval/funtrivia_questions_all.csv"),
    Path("eval/funtrivia_questions_all_200.csv"),
    Path("data/funtrivia_questions_all_200.csv"),
    Path("funtrivia_questions_all_200.csv"),
]

GOLD = next((p for p in candidates if p.exists()), None)
if GOLD is None:
    raise FileNotFoundError(f"None of expected funtrivia CSVs found. Looked for: {candidates}")

dfg = pd.read_csv(GOLD)

def contains_all(text: str, must_tokens: list[str]) -> bool:
    t = text.lower() if isinstance(text, str) else ""
    return all(tok.lower() in t for tok in must_tokens)

def recall_at_k_eval(df_gold, k=10):
    hits, total, rows = 0, 0, []
    # robustly find question / entities / id columns
    qcol = "question" if "question" in df_gold.columns else ("question_text" if "question_text" in df_gold.columns else next((c for c in df_gold.columns if "question" in c.lower()), None))
    entcol = "entities" if "entities" in df_gold.columns else None
    idcol = "question_id" if "question_id" in df_gold.columns else None

    if qcol is None:
        raise AssertionError("No question column found in gold CSV (expected 'question' or 'question_text').")

    # ensure search_fn exists
    if "search_fn" not in globals():
        raise NameError("search_fn is not defined. Run the retrieval/hybrid_search cell first.")

    for idx, row in df_gold.iterrows():
        q = str(row.get(qcol, "")).strip() if pd.notna(row.get(qcol, "")) else ""
        ents = []
        if entcol and pd.notna(row.get(entcol, "")):
            ents = [t.strip() for t in str(row[entcol]).split(";") if t.strip()]

        # call search function (must return DataFrame with 'text' column)
        res = search_fn(q, topk=k)
        joined = " ".join(res["text"].tolist()) if (hasattr(res, "__len__") and len(res)) else ""
        ok = contains_all(joined, ents) if ents else (len(res) > 0)
        hits += int(ok); total += 1
        qid = row.get(idcol, idx) if idcol or True else idx
        rows.append({"question_id": qid, "ok": ok, "retrieved": len(res)})

    out = pd.DataFrame(rows)
    rerank_flag = "on" if globals().get("USE_RERANK", False) else "off"
    print(f"Retrieval Recall@{k}: {hits}/{total} = {hits/total:.2f}  |  Rerank={rerank_flag}")
    return out

eval_rows = recall_at_k_eval(dfg, k=10)
eval_rows.head()
# ...existing code...

# ...existing code...
import pandas as pd
from pathlib import Path

# Accept the new filename (primary) and fallbacks
candidates = [
    Path("eval/funtrivia_questions_all.csv"),
    Path("eval/funtrivia_questions_all_200.csv"),
    Path("data/funtrivia_questions_all_200.csv"),
    Path("funtrivia_questions_all_200.csv"),
]

GOLD = next((p for p in candidates if p.exists()), None)
if GOLD is None:
    raise FileNotFoundError(f"None of expected funtrivia CSVs found. Looked for: {candidates}")

dfg = pd.read_csv(GOLD)

def contains_all(text: str, must_tokens: list[str]) -> bool:
    t = text.lower() if isinstance(text, str) else ""
    return all(tok.lower() in t for tok in must_tokens)

def recall_at_k_eval(df_gold, k=10):
    hits, total, rows = 0, 0, []
    # robustly find question / entities / id columns
    qcol = "question" if "question" in df_gold.columns else ("question_text" if "question_text" in df_gold.columns else next((c for c in df_gold.columns if "question" in c.lower()), None))
    entcol = "entities" if "entities" in df_gold.columns else None
    idcol = "question_id" if "question_id" in df_gold.columns else None

    if qcol is None:
        raise AssertionError("No question column found in gold CSV (expected 'question' or 'question_text').")

    # ensure search_fn exists
    if "search_fn" not in globals():
        raise NameError("search_fn is not defined. Run the retrieval/hybrid_search cell first.")

    for idx, row in df_gold.iterrows():
        q = str(row.get(qcol, "")).strip() if pd.notna(row.get(qcol, "")) else ""
        ents = []
        if entcol and pd.notna(row.get(entcol, "")):
            ents = [t.strip() for t in str(row[entcol]).split(";") if t.strip()]

        # call search function (must return DataFrame with 'text' column)
        res = search_fn(q, topk=k)
        joined = " ".join(res["text"].tolist()) if (hasattr(res, "__len__") and len(res)) else ""
        ok = contains_all(joined, ents) if ents else (len(res) > 0)
        hits += int(ok); total += 1
        qid = row.get(idcol, idx) if idcol or True else idx
        rows.append({"question_id": qid, "ok": ok, "retrieved": len(res)})

    out = pd.DataFrame(rows)
    rerank_flag = "on" if globals().get("USE_RERANK", False) else "off"
    print(f"Retrieval Recall@{k}: {hits}/{total} = {hits/total:.2f}  |  Rerank={rerank_flag}")
    return out

eval_rows = recall_at_k_eval(dfg, k=10)
eval_rows.head()
# ...existing code...

FileNotFoundError: None of expected funtrivia CSVs found. Looked for: [PosixPath('eval/funtrivia_questions_all.csv'), PosixPath('eval/funtrivia_questions_all_200.csv'), PosixPath('data/funtrivia_questions_all_200.csv'), PosixPath('funtrivia_questions_all_200.csv')]

In [24]:
# ==========================
# Phase 3-B — Evaluate FunTrivia Gold Set
# ==========================
import pandas as pd, re
from collections import Counter
from pathlib import Path

GOLD_PATH = Path("eval/funtrivia_golden_set.csv")   # adjust if needed
dfg = pd.read_csv(GOLD_PATH)
dfg = dfg.head(5)

# confirm structure
print(dfg.columns)
print(f"Loaded {len(dfg)} trivia questions\n")

# ---- Normalization helpers ----
def normalize(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = s.lower()
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def em(pred: str, gold: str) -> float:
    return float(normalize(pred) == normalize(gold))

def f1(pred: str, gold: str) -> float:
    p = normalize(pred).split()
    g = normalize(gold).split()
    if not p or not g:
        return 0.0
    pc, gc = Counter(p), Counter(g)
    overlap = sum((pc & gc).values())
    if overlap == 0:
        return 0.0
    precision = overlap / len(p)
    recall = overlap / len(g)
    return 2 * precision * recall / (precision + recall)

# ---- Evaluate ----
results = []
subset = dfg.head(25)   # start small (25) to verify; increase later

for i, row in subset.iterrows():
    q = str(row["question"])
    gold = str(row["answer_short"])
    try:
        pred, hits = generate_answer(q)
        score_em = em(pred, gold)
        score_f1 = f1(pred, gold)
        results.append({
            "qnum": row["qnum"],
            "question": q,
            "gold_answer": gold,
            "pred_answer": pred,
            "em": score_em,
            "f1": score_f1,
            "source_url": row.get("source_url", "")
        })
        print(f"[{i+1}/{len(subset)}] EM={score_em:.2f} F1={score_f1:.2f}")
    except Exception as e:
        print(f"[{i+1}] ERROR: {e}")
        results.append({
            "qnum": row["qnum"],
            "question": q,
            "gold_answer": gold,
            "pred_answer": None,
            "em": 0.0,
            "f1": 0.0,
            "source_url": row.get("source_url", "")
        })

df_eval = pd.DataFrame(results)
avg_em = df_eval["em"].mean()
avg_f1 = df_eval["f1"].mean()

print("\n=== Aggregate Scores ===")
print(f"Exact Match (EM): {avg_em:.3f}")
print(f"F1 Score: {avg_f1:.3f}")

EVAL_OUT = Path("eval/funtrivia_eval_results.csv")
df_eval.to_csv(EVAL_OUT, index=False)
print(f"\nSaved detailed results to {EVAL_OUT}")

Index(['qnum', 'question', 'answer_short', 'explanation', 'source_url'], dtype='object')
Loaded 5 trivia questions

[1/5] EM=0.00 F1=0.00
[2/5] EM=0.00 F1=0.11
[3/5] EM=0.00 F1=0.00
[4/5] EM=0.00 F1=0.38
[5/5] EM=0.00 F1=0.00

=== Aggregate Scores ===
Exact Match (EM): 0.000
F1 Score: 0.096

Saved detailed results to eval/funtrivia_eval_results.csv


In [25]:
# ==========================
# Single-question test cell
# ==========================
import textwrap

# Pick one question to test
row = dfg.iloc[2]   # change index if you want another (0-based)
qnum = row["qnum"]
qtext = row["question"]
gold_answer = row["answer_short"]

print(f"\n=== Q{qnum}: {qtext}\n")

# Run your full pipeline (retrieval + OpenAI)
pred_answer, hits = generate_answer(qtext)

# Display
print("🤖 OpenAI + RAG Answer:")
print(textwrap.fill(pred_answer, width=100))

print("\n🏆 Gold (True) Answer:")
print(textwrap.fill(gold_answer, width=100))

# Optional: show top retrieved evidence
print("\n📚 Top Retrieved Evidence:")
for i, r in hits.head(3).iterrows():
    print(f"  S{int(r['season'])}E{int(r['episode'])} {r['t_start']:.0f}-{r['t_end']:.0f}s: {r['text'][:200]}")


=== Q3: The Night King has so far been the main villain of the series. At the conclusion of season 7, he destroys part of the Wall. He is able to accomplish this by using one of Daenerys' dragons to burn a hole in the wall. Which dragon does he kill?

🤖 OpenAI + RAG Answer:
I can’t answer confidently from the provided episodes.   Citations: S7E5 8-22s, S7E3 622-638s, S7E5
15-30s, S5E7 188-202s, S7E1 352-368s, S7E2 810-825s.

🏆 Gold (True) Answer:
Viserion

📚 Top Retrieved Evidence:
  S7E5 8-22s: The Wall. The only thing standing between us and the army of the dead is the Wall, and the Wall hasn't been properly manned in centuries. Last time we saw The Night King, it was at Hardhome.
  S7E3 622-638s: But if destiny has brought Daenerys Targaryen back to our shores, it has also made Jon Snow King in the North. You were the first to bring Dothraki to Westeros? He is the first to make allies of wildl
  S7E5 15-30s: and the Wall hasn't been properly manned in centuries. Last time we saw Th

In [2]:
!pip install datasets

  Using cached dill-0.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.6.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (13 kB)
  Using cached multiprocess-0.70.18-py313-none-any.whl.metadata (7.2 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached frozenlist-1.8.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (20 kB)
  Using cached multidict-6.7.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (5.3 kB)
  Using cached propcache-0.4.1-cp313-cp313-macosx_11_0_arm64.whl.metadata (13 kB)
  Using cached yarl-1.22.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (75 kB)
Using cached dill-0.4.0-py3-none-any.whl (119 kB)
Using cached multidict-6.7.0-cp313-cp313-macosx_11_0_arm64.whl (43 kB)
Using cached yarl-1.22.0-cp313-cp313-macosx_11_0_arm64.whl (93 kB)
Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl (15 kB)
Using cached aiosignal-1.4.0-py3-none-any.whl (7.5 kB)
Using cached frozen

In [27]:
# ==========================
# Phase 1-B — Enrich with Tuana/game-of-thrones (character lore)
# ==========================
from datasets import load_dataset
import pandas as pd
import ast

print("📥 Downloading dataset from Hugging Face (Tuana/game-of-thrones)...")
ds = load_dataset("Tuana/game-of-thrones", split="train")

# Convert to DataFrame
df_hf = ds.to_pandas()

# ---- Parse metadata safely ----
def extract_name(meta):
    if isinstance(meta, str):
        try:
            meta = ast.literal_eval(meta)
        except Exception:
            return None
    if isinstance(meta, dict):
        return meta.get("name", None)
    return None

df_hf["meta_name"] = df_hf["meta"].apply(extract_name)

# ---- Clean up ----
df_hf = df_hf.rename(columns={"content": "text"})
df_hf["chunk_kind"] = "character_lore"
df_hf["season"] = None
df_hf["episode"] = None
df_hf["t_start"] = 0.0
df_hf["t_end"] = 0.0

# Drop any empty texts
df_hf = df_hf.dropna(subset=["text"]).reset_index(drop=True)

print(f"✅ Loaded {len(df_hf)} lore/text chunks from Hugging Face")
print(df_hf.head(3)[["meta_name","text"]])

# ---- Merge with your existing subtitle dataframe ----
df_aug = pd.concat(
    [df_chunks, df_hf[["season","episode","t_start","t_end","text","chunk_kind"]]],
    ignore_index=True
)
print(f"🔗 Merged dataset now has {len(df_aug)} total chunks")

# ---- Optional snapshot ----
df_aug.to_csv("data/got_augmented_lore.csv", index=False)
print("💾 Saved enriched dataset to data/got_augmented_lore.csv")

📥 Downloading dataset from Hugging Face (Tuana/game-of-thrones)...
✅ Loaded 2357 lore/text chunks from Hugging Face
                                        meta_name  \
0  145_Elio_M._García_Jr._and_Linda_Antonsson.txt   
1  145_Elio_M._García_Jr._and_Linda_Antonsson.txt   
2  145_Elio_M._García_Jr._and_Linda_Antonsson.txt   

                                                text  
0  Linda Antonsson and Elio García at Archipelaco...  
1  \n==Career==\nElio García was attending the Un...  
2  \n==Personal life==\nGarcía and Antonsson met ...  
🔗 Merged dataset now has 17414 total chunks
💾 Saved enriched dataset to data/got_augmented_lore.csv


In [31]:
# ==========================
# Phase 1-C — Re-Embedding (Subtitles + Lore)
# ==========================
# ...existing code...
import numpy as np, faiss
from tqdm import tqdm
from rank_bm25 import BM25Okapi
from pathlib import Path

# prefer loading saved augmented artifacts if present
OUT_VEC = Path("data/got_aug_vecs.npy")
OUT_INDEX = Path("data/got_aug_faiss.bin")
OUT_DF   = Path("data/got_aug_chunks.csv")

# ensure df_aug exists (try memory first, then snapshot)
if "df_aug" not in globals() or df_aug is None:
    if OUT_DF.exists():
        df_aug = pd.read_csv(OUT_DF)
        print(f"Loaded df_aug from {OUT_DF} ({len(df_aug)} rows)")
    else:
        raise FileNotFoundError("df_aug not in memory and no snapshot at data/got_aug_chunks.csv. Run Phase 1-B first.")

print(f"Total augmented chunks: {len(df_aug)}")

# helper embedder (reuse safe batching)
EMBED_MODEL = "text-embedding-3-large"
def embed_texts(texts, batch_size=16, max_chars=8000):
    import time
    vecs = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Embedding"):
        batch = texts[i:i+batch_size]
        batch_trunc = [t if len(t) <= max_chars else t[:max_chars] + " [TRUNCATED]" for t in batch]
        try:
            resp = client.embeddings.create(model=EMBED_MODEL, input=batch_trunc)
            vecs.extend([e.embedding for e in resp.data])
        except Exception as e:
            print(f"Batch embed failed at idx {i}: {e}. Retrying individually.")
            for t in batch_trunc:
                for attempt in range(3):
                    try:
                        r = client.embeddings.create(model=EMBED_MODEL, input=[t])
                        vecs.extend([e.embedding for e in r.data])
                        break
                    except Exception as e2:
                        print(f"  attempt {attempt+1} failed: {e2}")
                        time.sleep(1 + attempt * 2)
                else:
                    raise RuntimeError("Repeated embedding failure; reduce max_chars.")
    return np.asarray(vecs, dtype="float32")

# load or compute vectors + FAISS index
if OUT_VEC.exists() and OUT_INDEX.exists():
    print("Loading saved augmented vectors and FAISS index...")
    vecs = np.load(OUT_VEC)
    try:
        index = faiss.read_index(str(OUT_INDEX))
    except Exception as e:
        print(f"Failed to read FAISS index: {e}. Rebuilding index from vectors.")
        faiss.normalize_L2(vecs)
        index = faiss.IndexFlatIP(vecs.shape[1])
        index.add(vecs)
        faiss.write_index(index, str(OUT_INDEX))
    print(f"Loaded vectors ({OUT_VEC}) and index ({OUT_INDEX}) — ntotal={index.ntotal}")
else:
    print("No saved augmented artifacts found — computing embeddings and building FAISS index.")
    vecs = embed_texts(df_aug["text"].fillna("").tolist())
    faiss.normalize_L2(vecs)
    index = faiss.IndexFlatIP(vecs.shape[1])
    index.add(vecs)
    OUT_VEC.parent.mkdir(parents=True, exist_ok=True)
    np.save(OUT_VEC, vecs)
    faiss.write_index(index, str(OUT_INDEX))
    df_aug.to_csv(OUT_DF, index=False)
    print(f"Saved vectors -> {OUT_VEC} and FAISS index -> {OUT_INDEX}")

print(f"FAISS index ready: {index.ntotal} vectors")
# ...existing code...
# Replace the "Tokenizing for BM25 (df_aug)..." block with cached-load logic
OUT_TOK = Path("data/got_aug_corpus_tokens.pkl")
import pickle

# try to load cached tokens to avoid re-tokenizing
if OUT_TOK.exists():
    with open(OUT_TOK, "rb") as f:
        corpus_tokens = pickle.load(f)
    print(f"Loaded BM25 corpus tokens from {OUT_TOK} ({len(corpus_tokens)} docs)")
else:
    print("Tokenizing for BM25 (df_aug)...")
    corpus_tokens = [[tok.text.lower() for tok in nlp(txt)] for txt in tqdm(df_aug["text"].fillna(""), desc="Tokenizing")]
    OUT_TOK.parent.mkdir(parents=True, exist_ok=True)
    with open(OUT_TOK, "wb") as f:
        pickle.dump(corpus_tokens, f)
    print(f"Saved BM25 corpus tokens -> {OUT_TOK}")

bm25 = BM25Okapi(corpus_tokens)
print("BM25 ready (df_aug)")
# ...existing code...

# quick sanity retrieval function that uses df_aug/index/bm25
def hybrid_search_aug(query: str, topk: int = 10, alpha: float = 0.35, cand_mult: int = 20):
    import numpy as np, pandas as pd
    q_emb = client.embeddings.create(model=EMBED_MODEL, input=[query]).data[0].embedding
    qv = np.asarray(q_emb, dtype="float32")[None, :]
    faiss.normalize_L2(qv)
    D, I = index.search(qv, topk * cand_mult)
    vec_scores, vec_idx = D[0].tolist(), I[0].tolist()

    q_tokens = [t.text.lower() for t in nlp(query)]
    bm_scores = bm25.get_scores(q_tokens)
    bm_top = np.argsort(bm_scores)[::-1][:topk * cand_mult]

    max_valid = len(df_aug)
    # filter indices to valid rows
    valid_vec_pairs = [(int(idx), float(score)) for idx, score in zip(vec_idx, vec_scores)
                       if isinstance(idx, (int, np.integer)) and 0 <= int(idx) < max_valid]
    v_map = {i: s for i, s in valid_vec_pairs}
    vec_idx_valid = [i for i, _ in valid_vec_pairs]
    bm_top_valid = [int(i) for i in bm_top if 0 <= int(i) < max_valid]

    cand = list(set(vec_idx_valid) | set(bm_top_valid))
    if not cand:
        return pd.DataFrame([])

    bm_max = max(bm_scores) if len(bm_scores) else 1.0
    scored = []
    for i in cand:
        v = float(v_map.get(int(i), 0.0))
        b = float(bm_scores[int(i)]) / (bm_max + 1e-6)
        scored.append((int(i), alpha * v + (1 - alpha) * b))
    scored.sort(key=lambda x: x[1], reverse=True)

    rows = []
    for i, sc in scored[:topk]:
        r = df_aug.iloc[int(i)].to_dict()
        r["score"] = float(sc)
        rows.append(r)
    return pd.DataFrame(rows)
# ...existing code...

print("\n🔍 Test retrieval:")
res = hybrid_search_aug("Which dragon does the Night King kill?", topk=5)
print(res[["chunk_kind","score"]])
print("\nTop hit sample:\n", res.iloc[0]["text"][:250])

Total augmented chunks: 17414
Loading saved augmented vectors and FAISS index...
Loaded vectors (data/got_aug_vecs.npy) and index (data/got_aug_faiss.bin) — ntotal=17414
FAISS index ready: 17414 vectors
Tokenizing for BM25 (df_aug)...


Tokenizing: 100%|██████████| 17414/17414 [06:14<00:00, 46.49it/s] 


Saved BM25 corpus tokens -> data/got_aug_corpus_tokens.pkl
BM25 ready (df_aug)

🔍 Test retrieval:
       chunk_kind     score
0  character_lore  0.870180
1  character_lore  0.790736
2          window  0.774781
3  character_lore  0.770592
4  character_lore  0.770233

Top hit sample:
 
===Season 8===
Led by the Night King, the dead quickly advance south and attack Last Hearth, massacring the population and adding them to their army, before marching on Winterfell. Bran, knowing that the Night King plans to destroy him, plans to hid


In [6]:
# Reload the FunTrivia golden set
import pandas as pd
from pathlib import Path

GOLD_PATH = Path("eval/funtrivia_golden_set.csv")  # adjust if needed
dfg = pd.read_csv(GOLD_PATH)

print(f"Loaded {len(dfg)} trivia questions")
print(dfg.head(3)[["qnum","question","answer_short"]])

Loaded 165 trivia questions
   qnum                                           question     answer_short
0     1  Who are the two brothers of the Night's Watch ...  Brant and Derek
1     2  Daenerys is traded away as a bride to a "savag...          Viserys
2     3  The Night King has so far been the main villai...         Viserion


In [7]:

ANSWER_PROMPT = """You are a Game of Thrones expert.
Use the retrieved evidence below to answer the user's question truthfully and concisely.
Cite evidence explicitly if possible.

Question: {question}

Evidence:
{evidence}
"""

GEN_MODEL = "gpt-4o-mini"   # or whichever model you want

def format_evidence_rows(df):
    out_lines = []
    for _, r in df.head(5).iterrows():
        s = r.get("season", "")
        e = r.get("episode", "")
        t = f"S{s}E{e}" if pd.notna(s) and pd.notna(e) else ""
        txt = r["text"].strip().replace("\n", " ")
        out_lines.append(f"[{t}] {txt}")
    return "\n".join(out_lines)

def generate_answer(question, topk=10):
    hits = hybrid_search_aug(question, topk=topk)  # <-- use augmented search
    ev_text = format_evidence_rows(hits)
    prompt = ANSWER_PROMPT.format(question=question, evidence=ev_text)
    out = client.chat.completions.create(
        model=GEN_MODEL,
        temperature=0.2,
        messages=[{"role": "user", "content": prompt}]
    )
    answer = out.choices[0].message.content.strip()
    return answer, hits

# ==========================
# Single-question test cell
# ==========================
import textwrap

# Pick one question to test
row = dfg.iloc[1]   # change index if you want another (0-based)
qnum = row["qnum"]
qtext = row["question"]
gold_answer = row["answer_short"]

print(f"\n=== Q{qnum}: {qtext}\n")

# Run your full pipeline (retrieval + OpenAI)
pred_answer, hits = generate_answer(qtext)

# Display
print("🤖 OpenAI + RAG Answer:")
print(textwrap.fill(pred_answer, width=100))

print("\n🏆 Gold (True) Answer:")
print(textwrap.fill(gold_answer, width=100))

# Optional: show top retrieved evidence
print("\n📚 Top Retrieved Evidence:")
for _, r in hits.head(3).iterrows():
    s = r.get("season")
    e = r.get("episode")
    # gracefully handle NaN or None
    s_label = f"S{int(s)}" if pd.notna(s) else "S?"
    e_label = f"E{int(e)}" if pd.notna(e) else "E?"
    t_label = f"{s_label}{e_label}"
    text_snip = r["text"][:200].replace("\n", " ")
    print(f"  {t_label} {r['t_start']:.0f}-{r['t_end']:.0f}s: {text_snip}")


=== Q2: Daenerys is traded away as a bride to a "savage" in season one. Who selfishly treats her like a means to an end?



NameError: name 'hybrid_search_aug' is not defined

In [21]:
import numpy as np
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def semantic_similarity(pred: str, gold: str) -> float:
    try:
        resp = client.embeddings.create(
            model="text-embedding-3-small",
            input=[pred, gold]
        )
        v_pred, v_gold = np.array(resp.data[0].embedding), np.array(resp.data[1].embedding)
        return float(cosine_similarity(v_pred, v_gold))
    except Exception as e:
        print("Embedding similarity error:", e)
        return 0.0

# compute all three
sem = semantic_similarity(pred_answer, gold_answer)
score_em = em(pred_answer, gold_answer)
score_f1 = f1(pred_answer, gold_answer)

print("\n📈 Evaluation Scores:")
print(f"  Exact Match (EM): {score_em:.2f}")
print(f"  F1 Score:         {score_f1:.2f}")
print(f"  Semantic Similarity: {sem:.2f}")


📈 Evaluation Scores:
  Exact Match (EM): 0.00
  F1 Score:         0.02
  Semantic Similarity: 0.42


In [20]:
import re
from collections import Counter

def normalize(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = s.lower()
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def em(pred: str, gold: str) -> float:
    return float(normalize(pred) == normalize(gold))

def f1(pred: str, gold: str) -> float:
    p = normalize(pred).split()
    g = normalize(gold).split()
    if not p or not g:
        return 0.0
    pc, gc = Counter(p), Counter(g)
    overlap = sum((pc & gc).values())
    if overlap == 0:
        return 0.0
    precision = overlap / len(p)
    recall = overlap / len(g)
    return 2 * precision * recall / (precision + recall)

# Compute and print scores
score_em = em(pred_answer, gold_answer)
score_f1 = f1(pred_answer, gold_answer)

print("\n📈 Evaluation Scores:")
print(f"  Exact Match (EM): {score_em:.2f}")
print(f"  F1 Score:         {score_f1:.2f}")


📈 Evaluation Scores:
  Exact Match (EM): 0.00
  F1 Score:         0.02


In [22]:
# ==========================
# Phase 3-B — Full Evaluation (EM, F1, Semantic)
# ==========================
import pandas as pd, re, numpy as np
from collections import Counter
from pathlib import Path
from openai import OpenAI
from tqdm import tqdm

# --- Setup ---
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
GEN_MODEL = "gpt-4o-mini"
GOLD_PATH = Path("eval/funtrivia_golden_set.csv")
EVAL_OUT = Path("eval/funtrivia_eval_full.csv")

# --- Load data ---
dfg = pd.read_csv(GOLD_PATH)
print(f"Loaded {len(dfg)} gold questions")

# --- Helpers ---
def normalize(s: str) -> str:
    if not isinstance(s, str): return ""
    s = s.lower()
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def em(pred, gold):
    return float(normalize(pred) == normalize(gold))

def f1(pred, gold):
    p = normalize(pred).split()
    g = normalize(gold).split()
    if not p or not g:
        return 0.0
    pc, gc = Counter(p), Counter(g)
    overlap = sum((pc & gc).values())
    if overlap == 0:
        return 0.0
    precision = overlap / len(p)
    recall = overlap / len(g)
    return 2 * precision * recall / (precision + recall)

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def semantic_similarity(pred, gold):
    try:
        resp = client.embeddings.create(model="text-embedding-3-small", input=[pred, gold])
        v_pred, v_gold = np.array(resp.data[0].embedding), np.array(resp.data[1].embedding)
        return float(cosine_similarity(v_pred, v_gold))
    except Exception as e:
        print("Embedding similarity error:", e)
        return 0.0

def format_evidence_rows(df):
    out_lines = []
    for _, r in df.head(5).iterrows():
        s = r.get("season", "?")
        e = r.get("episode", "?")
        t = f"S{s}E{e}" if pd.notna(s) and pd.notna(e) else ""
        txt = str(r.get("text", "")).strip().replace("\n", " ")
        out_lines.append(f"[{t}] {txt}")
    return "\n".join(out_lines)

ANSWER_PROMPT = """You are a Game of Thrones expert.
Use the retrieved evidence below to answer truthfully and concisely.
Cite the season/episode if possible.

Question: {question}

Evidence:
{evidence}
"""

def generate_answer(question, topk=10):
    hits = hybrid_search_aug(question, topk=topk)  # <-- use your augmented retriever
    ev_text = format_evidence_rows(hits)
    prompt = ANSWER_PROMPT.format(question=question, evidence=ev_text)
    out = client.chat.completions.create(
        model=GEN_MODEL,
        temperature=0.2,
        messages=[{"role": "user", "content": prompt}]
    )
    answer = out.choices[0].message.content.strip()
    return answer, hits

# --- Evaluation loop ---
results = []
for i, row in tqdm(dfg.iterrows(), total=len(dfg), desc="Evaluating"):
    q = str(row["question"])
    gold = str(row["answer_short"])
    try:
        pred, hits = generate_answer(q)
        score_em = em(pred, gold)
        score_f1 = f1(pred, gold)
        score_sem = semantic_similarity(pred, gold)
        results.append({
            "qnum": row["qnum"],
            "question": q,
            "gold_answer": gold,
            "pred_answer": pred,
            "em": score_em,
            "f1": score_f1,
            "semantic": score_sem
        })
    except Exception as e:
        print(f"[Error on Q{row['qnum']}] {e}")
        results.append({
            "qnum": row["qnum"],
            "question": q,
            "gold_answer": gold,
            "pred_answer": None,
            "em": 0.0,
            "f1": 0.0,
            "semantic": 0.0
        })

df_eval = pd.DataFrame(results)
df_eval.to_csv(EVAL_OUT, index=False)

# --- Aggregate metrics ---
avg_em = df_eval["em"].mean()
avg_f1 = df_eval["f1"].mean()
avg_sem = df_eval["semantic"].mean()

print("\n📈 Evaluation Summary")
print(f"Exact Match (EM):       {avg_em:.3f}")
print(f"F1 Score:               {avg_f1:.3f}")
print(f"Semantic Similarity:    {avg_sem:.3f}")
print(f"Saved detailed results → {EVAL_OUT}")

Loaded 165 gold questions


Evaluating: 100%|██████████| 165/165 [07:13<00:00,  2.63s/it]


📈 Evaluation Summary
Exact Match (EM):       0.000
F1 Score:               0.072
Semantic Similarity:    0.416
Saved detailed results → eval/funtrivia_eval_full.csv


In [33]:
# language: python
# Quick "load everything" helper to skip embedding / tokenizing steps.
# Run this cell before the nodes cell.

import os, pickle
from pathlib import Path
import numpy as np
import pandas as pd

# ...existing code...
def hybrid_search_aug(query: str, topk: int = 10, alpha: float = 0.35, cand_mult: int = 20):
    import numpy as np, pandas as pd
    # require loader cell to have set: df_aug, index, bm25, client, nlp, EMBED_MODEL
    if "df_aug" not in globals():
        raise NameError("df_aug not found. Run the loader cell that loads precomputed artifacts first.")
    q_emb = client.embeddings.create(model=EMBED_MODEL, input=[query]).data[0].embedding
    qv = np.asarray(q_emb, dtype="float32")[None, :]
    faiss.normalize_L2(qv)
    D, I = index.search(qv, topk * cand_mult)
    vec_scores, vec_idx = D[0].tolist(), I[0].tolist()

    q_tokens = [t.text.lower() for t in nlp(query)]
    bm_scores = bm25.get_scores(q_tokens)
    bm_top = np.argsort(bm_scores)[::-1][:topk * cand_mult]

    max_valid = len(df_aug)
    valid_vec_pairs = [
        (int(idx), float(score))
        for idx, score in zip(vec_idx, vec_scores)
        if isinstance(idx, (int, np.integer)) and 0 <= int(idx) < max_valid
    ]
    v_map = {i: s for i, s in valid_vec_pairs}
    vec_idx_valid = [i for i, _ in valid_vec_pairs]
    bm_top_valid = [int(i) for i in bm_top if 0 <= int(i) < max_valid]

    cand = list(set(vec_idx_valid) | set(bm_top_valid))
    if not cand:
        return pd.DataFrame([])

    bm_max = max(bm_scores) if len(bm_scores) else 1.0
    scored = []
    for i in cand:
        v = float(v_map.get(int(i), 0.0))
        b = float(bm_scores[int(i)]) / (bm_max + 1e-6)
        scored.append((int(i), alpha * v + (1 - alpha) * b))
    scored.sort(key=lambda x: x[1], reverse=True)

    rows = []
    for i, sc in scored[:topk]:
        r = df_aug.iloc[int(i)].to_dict()
        r["score"] = float(sc)
        rows.append(r)
    return pd.DataFrame(rows)
# ...existing code...

# OpenAI client (same API used in notebook)
try:
    from openai import OpenAI
except Exception:
    raise RuntimeError("openai package or OpenAI import not available. Install 'openai' and retry.")
OPENAI_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_KEY:
    raise RuntimeError("OPENAI_API_KEY not set in environment. Set it or run dotenv load step.")
client = OpenAI(api_key=OPENAI_KEY)

# spaCy
try:
    import spacy
    nlp = spacy.load("en_core_web_sm")
except Exception as e:
    raise RuntimeError(f"spaCy model not ready: {e}. Run `python -m spacy download en_core_web_sm` if needed.")

# BM25 and FAISS
try:
    from rank_bm25 import BM25Okapi
except Exception:
    raise RuntimeError("rank_bm25 not installed. pip install rank-bm25")

try:
    import faiss
except Exception:
    raise RuntimeError("faiss-cpu not installed. pip install faiss-cpu")

# Candidate artifact paths (prefer augmented artifacts)
BASE = Path("data")
DF_AUG_PATHS = [BASE / "got_aug_chunks.csv", BASE / "got_augmented_lore.csv", BASE / "got_aug_chunks.csv"]
DF_CHUNK_PATHS = [BASE / "got_chunks.csv", BASE / "got_chunks.csv", BASE / "got_chunks.csv"]
VEC_PATHS = [BASE / "got_aug_vecs.npy", BASE / "got_vecs.npy", BASE / "got_chunks_vecs.npy"]
IDX_PATHS = [BASE / "got_aug_faiss.bin", BASE / "got_faiss.bin", BASE / "got_chunks_faiss.bin"]
TOK_PATHS = [BASE / "got_aug_corpus_tokens.pkl", BASE / "got_corpus_tokens.pkl", BASE / "got_corpus_tokens.pkl"]

# load dataframe (prefer augmented)
df_aug = None
for p in DF_AUG_PATHS + DF_CHUNK_PATHS:
    if p.exists():
        df_aug = pd.read_csv(p)
        print(f"Loaded dataframe from {p} ({len(df_aug)} rows)")
        break
if df_aug is None:
    raise FileNotFoundError("No chunk dataframe found. Expected one of: " + ", ".join(str(p) for p in DF_AUG_PATHS + DF_CHUNK_PATHS))

# load vectors & index
vec_path = next((p for p in VEC_PATHS if p.exists()), None)
idx_path = next((p for p in IDX_PATHS if p.exists()), None)

if vec_path is None and idx_path is None:
    raise FileNotFoundError("No precomputed vectors or FAISS index found in data/. Place got_aug_vecs.npy/got_aug_faiss.bin (or equivalents) in data/")

if vec_path is not None:
    vecs = np.load(vec_path)
    print(f"Loaded vectors from {vec_path} ({vecs.shape})")
else:
    vecs = None

if idx_path is not None:
    try:
        index = faiss.read_index(str(idx_path))
        print(f"Loaded FAISS index from {idx_path} (ntotal={index.ntotal})")
    except Exception as e:
        if vecs is None:
            raise RuntimeError(f"Failed to read FAISS index and no vectors to rebuild: {e}")
        faiss.normalize_L2(vecs)
        index = faiss.IndexFlatIP(vecs.shape[1])
        index.add(vecs)
        print("Rebuilt FAISS index from vectors")
else:
    # build index from vecs
    if vecs is None:
        raise FileNotFoundError("No FAISS index path and no vector file to build from.")
    faiss.normalize_L2(vecs)
    index = faiss.IndexFlatIP(vecs.shape[1])
    index.add(vecs)
    print("Built FAISS index from vectors (no index file present)")

# load BM25 token cache
tok_path = next((p for p in TOK_PATHS if p.exists()), None)
corpus_tokens = None
if tok_path:
    with open(tok_path, "rb") as f:
        corpus_tokens = pickle.load(f)
    print(f"Loaded BM25 token cache from {tok_path} ({len(corpus_tokens)} docs)")
else:
    raise FileNotFoundError("BM25 token cache not found (expected data/got_aug_corpus_tokens.pkl or similar). If you want to rebuild tokens run the tokenization cell once.")

# init BM25
bm25 = BM25Okapi(corpus_tokens)
print("BM25 index ready")

# set standard globals used by notebook cells
EMBED_MODEL = globals().get("EMBED_MODEL", "text-embedding-3-large")
_GOT_RETRIEVAL_SETUP_DONE = True
_bm25_row_count = len(df_aug)

# optional reranker placeholder (keeps behavior consistent)
RERANKER = None
try:
    from sentence_transformers import CrossEncoder
    RERANKER = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
    print("Loaded CrossEncoder reranker")
except Exception:
    print("CrossEncoder reranker not loaded (optional)")

print("\nSetup complete. Globals available: client, nlp, df_aug, index, bm25, EMBED_MODEL")

Loaded dataframe from data/got_aug_chunks.csv (17414 rows)
Loaded vectors from data/got_aug_vecs.npy ((17414, 3072))
Loaded FAISS index from data/got_aug_faiss.bin (ntotal=17414)
Loaded BM25 token cache from data/got_aug_corpus_tokens.pkl (17414 docs)
BM25 index ready
Loaded CrossEncoder reranker

Setup complete. Globals available: client, nlp, df_aug, index, bm25, EMBED_MODEL


In [34]:
# ==========================
# COSINE of Thrones — Phase 3 Agentic RAG Scaffold (LangGraph-style)
# ==========================
# Prereqs expected in memory:
# - nlp (spaCy English model)
# - client = OpenAI(...)
# - hybrid_search_aug(query, topk=10, alpha=0.6, cand_mult=20, use_ner_boost=True)
# - GEN_MODEL (e.g., "gpt-4o-mini")
# - ANSWER_PROMPT (or we define a concise one below)

from dataclasses import dataclass, field
from typing import List, Dict, Any, Optional
import pandas as pd
import numpy as np



# ---- Optional cross-encoder reranker ----
RERANKER = None
try:
    from sentence_transformers import CrossEncoder
    RERANKER = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
    print("Reranker loaded: cross-encoder/ms-marco-MiniLM-L-6-v2")
except Exception as _e:
    print("Reranker not available; continuing without cross-encoder.")

# ---- Minimal prompt (concise, citation-first) ----
ANSWER_PROMPT = """You are a Game of Thrones expert.
Answer in one sentence, using ONLY the evidence lines below. If evidence is insufficient, say:
"I cannot find this in the provided evidence."
Include season/episode if present.

Question: {question}

Evidence:
{evidence}
"""

# ---- Shared helpers ----
def format_evidence_rows(df: pd.DataFrame, k: int = 5) -> str:
    if df is None or len(df) == 0:
        return "(no evidence)"
    lines = []
    for _, r in df.head(k).iterrows():
        s, e = r.get("season"), r.get("episode")
        tag = f"S{int(s)}E{int(e)}" if pd.notna(s) and pd.notna(e) else "S?E?"
        txt = str(r.get("text","")).replace("\n"," ").strip()
        spk = r.get("speaker")
        prefix = f"[{tag}] {spk}: " if isinstance(spk, str) and spk.strip() else f"[{tag}] "
        lines.append(prefix + txt)
    return "\n".join(lines)

def guess_question_type(q: str) -> str:
    ql = q.lower()
    for t in ["who","what","when","where","why","how","which"]:
        if ql.startswith(t) or f" {t} " in ql:
            return t
    return "open"

def extract_entities(q: str) -> List[str]:
    doc = nlp(q)
    return [ent.text for ent in doc.ents]

# ---- Graph State ----
@dataclass
class RAGState:
    question: str
    question_type: str = "open"
    entities: List[str] = field(default_factory=list)
    retrieved: Optional[pd.DataFrame] = None
    reranked: Optional[pd.DataFrame] = None
    evidence_text: str = ""
    answer: str = ""
    logs: Dict[str, Any] = field(default_factory=dict)

# ---- Nodes ----
def node_query_parser(state: RAGState) -> RAGState:
    ents = extract_entities(state.question)
    qtype = guess_question_type(state.question)
    state.entities = ents
    state.question_type = qtype
    state.logs["parser"] = {"entities": ents, "question_type": qtype}
    return state

def node_retriever(state: RAGState, topk: int = 12, alpha: float = 0.6) -> RAGState:
    # Uses your augmented hybrid search
    hits = hybrid_search_aug(state.question, topk=topk, alpha=alpha)
    state.retrieved = hits
    state.logs["retriever"] = {"n_hits": 0 if hits is None else int(len(hits))}
    return state

def node_reranker(state: RAGState) -> RAGState:
    if state.retrieved is None or len(state.retrieved) == 0 or RERANKER is None:
        # fall back to retrieved as-is
        state.reranked = state.retrieved
        state.logs["reranker"] = {"used": False, "reason": "no-hits-or-no-model"}
        return state

    pairs = [[state.question, t] for t in state.retrieved["text"].tolist()]
    scores = RERANKER.predict(pairs)
    df = state.retrieved.copy().reset_index(drop=True)
    df["rerank_score"] = scores
    df = df.sort_values("rerank_score", ascending=False).reset_index(drop=True)
    state.reranked = df
    state.logs["reranker"] = {"used": True, "top_score": float(df.iloc[0]["rerank_score"])}
    return state

def node_synthesizer(state: RAGState, k_evidence: int = 5, show_prompt: bool = True) -> RAGState:
    """
    Generate grounded answer from retrieved evidence.
    Optionally prints the full prompt (question + evidence) before sending to OpenAI.
    """
    hits = state.reranked if state.reranked is not None else state.retrieved
    ev_text = format_evidence_rows(hits, k=k_evidence)
    state.evidence_text = ev_text

    # ---- Construct the full prompt ----
    prompt = ANSWER_PROMPT.format(question=state.question, evidence=ev_text)

    # ---- Print for debugging ----
    if show_prompt:
        print("\n=======================")
        print("🔍 PROMPT SENT TO OPENAI")
        print("=======================\n")
        print(prompt)
        print("\n=======================\n")

    # ---- Call the model ----
    out = client.chat.completions.create(
        model=GEN_MODEL,
        temperature=0.1,
        messages=[{"role": "user", "content": prompt}]
    )

    state.answer = out.choices[0].message.content.strip()
    state.logs["synthesizer"] = {
        "prompt_length_chars": len(prompt),
        "evidence_count": len(hits) if hits is not None else 0
    }
    return state

# ---- Graph runner ----
def run_graph(question: str) -> RAGState:
    state = RAGState(question=question)
    # Ordered pipeline
    state = node_query_parser(state)
    state = node_retriever(state, topk=12, alpha=0.6)
    state = node_reranker(state)
    state = node_synthesizer(state, k_evidence=5)
    return state

# ==========================
# Smoke test
# ==========================
sample_q = "Who is Jon Snow's mother in Game of Thrones?"
st = run_graph(sample_q)

print("Q:", st.question)
print("\n--- Evidence ---")
print(st.evidence_text)
print("\n--- Answer ---")
print(st.answer)
print("\n--- Logs ---")
for k,v in st.logs.items():
    print(k, "=>", v)

Reranker loaded: cross-encoder/ms-marco-MiniLM-L-6-v2

🔍 PROMPT SENT TO OPENAI

You are a Game of Thrones expert.
Answer in one sentence, using ONLY the evidence lines below. If evidence is insufficient, say:
"I cannot find this in the provided evidence."
Include season/episode if present.

Question: Who is Jon Snow's mother in Game of Thrones?

Evidence:
[S?E?] ===Series reprisals=== * Jon Snow (Kit Harington), a member of the Night's Watch and bastard son of Ned Stark. * Cersei Lannister (Lena Headey), the Queen Regent of the Seven Kingdoms serving in King's Landing and mother to King Joffrey Baratheon. * Tyrion Lannister (Peter Dinklage), the Master of Coin serving in King's Landing and Queen Cersei's brother. * Ramsay Snow (Iwan Rheon), a sadistic and ruthless bastard son of Roose Bolton, the Warden of the North. * Daenerys Targaryen (Emilia Clarke), the Mother of Dragons and the potential future queen of the Seven Kingdoms, operating in Meereen. * Margaery Tyrell  (Natalie Dormer)

In [35]:
pip install scikit-learn

  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 26.6 MB/s  0:00:00 eta 0:00:01
Using cached joblib-1.5.2-py3-none-any.whl (308 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 38.5 MB/s  0:00:00 eta 0:00:01
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [scikit-learn] [scikit-learn]
Note: you may need to restart the kernel to use updated packages.


In [36]:
# ==========================
# COSINE of Thrones — Phase 3-B Full Evaluation
# ==========================
import pandas as pd, re, textwrap, time
from pathlib import Path
from collections import Counter
from tqdm import tqdm

GOLD_PATH = Path("eval/funtrivia_golden_set.csv")
dfg = pd.read_csv(GOLD_PATH)

print(f"Loaded {len(dfg)} golden questions")

# ---- Normalization & Scoring ----
def normalize(s: str) -> str:
    s = str(s).lower()
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    return re.sub(r"\s+", " ", s).strip()

def em(pred: str, gold: str) -> float:
    return float(normalize(pred) == normalize(gold))

def f1(pred: str, gold: str) -> float:
    p, g = normalize(pred).split(), normalize(gold).split()
    if not p or not g: return 0.0
    pc, gc = Counter(p), Counter(g)
    overlap = sum((pc & gc).values())
    if overlap == 0: return 0.0
    prec, rec = overlap/len(p), overlap/len(g)
    return 2*prec*rec/(prec+rec)

# Optional semantic similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
def semantic_sim(pred, gold):
    vec = TfidfVectorizer().fit_transform([normalize(pred), normalize(gold)])
    return float(cosine_similarity(vec)[0,1])

# ---- Evaluation loop ----
results = []
for i, row in tqdm(dfg.iterrows(), total=len(dfg)):
    q = str(row["question"])
    gold = str(row["answer_short"])
    try:
        st = run_graph(q)
        pred = st.answer
        score_em = em(pred, gold)
        score_f1 = f1(pred, gold)
        score_sem = semantic_sim(pred, gold)
        results.append({
            "qnum": row.get("qnum", i+1),
            "question": q,
            "gold": gold,
            "pred": pred,
            "em": score_em,
            "f1": score_f1,
            "semantic": score_sem,
            "entities": "; ".join(st.entities),
            "question_type": st.question_type,
            "evidence": st.evidence_text[:500],  # store short snippet
        })
    except Exception as e:
        print(f"[{i+1}] ERROR: {e}")
        results.append({
            "qnum": row.get("qnum", i+1),
            "question": q,
            "gold": gold,
            "pred": str(e),
            "em": 0.0,
            "f1": 0.0,
            "semantic": 0.0,
            "entities": "",
            "question_type": "",
            "evidence": "",
        })
    time.sleep(0.3)  # gentle rate limit

# ---- Results summary ----
df_eval = pd.DataFrame(results)
avg_em = df_eval["em"].mean()
avg_f1 = df_eval["f1"].mean()
avg_sem = df_eval["semantic"].mean()

print("\n📈 Evaluation Summary")
print(f"Exact Match (EM):       {avg_em:.3f}")
print(f"F1 Score:               {avg_f1:.3f}")
print(f"Semantic Similarity:    {avg_sem:.3f}")

OUT = Path("eval/funtrivia_agentic_eval.csv")
df_eval.to_csv(OUT, index=False)
print(f"Saved detailed results → {OUT}")

Loaded 165 golden questions


  0%|          | 0/165 [00:00<?, ?it/s]


🔍 PROMPT SENT TO OPENAI

You are a Game of Thrones expert.
Answer in one sentence, using ONLY the evidence lines below. If evidence is insufficient, say:
"I cannot find this in the provided evidence."
Include season/episode if present.

Question: Who are the two brothers of the Night's Watch Sam must fight to protect Gilly when she is nearly raped?

Evidence:
[S?E?] ====Season 5==== When the Night's Watch convenes to choose their new Lord Commander, Sam speaks up to nominate Jon, who is ultimately elected. Sam spends his time researching the White Walkers' weaknesses and tending to the dying Maester Aemon, and he comforts Aemon in his final moments as he dies of old age. Later, Sam is beaten while defending Gilly from two Night's Watchmen who attempt to rape her, but he and Gilly are saved by Jon's direwolf Ghost. Gilly tends to Sam's wounds, and the couple finally consummates their relationship. After Jon returns from Hardhome, Sam entreats him to send him to the Citadel in Oldtown i

  1%|          | 1/165 [00:03<10:20,  3.78s/it]


🔍 PROMPT SENT TO OPENAI

You are a Game of Thrones expert.
Answer in one sentence, using ONLY the evidence lines below. If evidence is insufficient, say:
"I cannot find this in the provided evidence."
Include season/episode if present.

Question: Daenerys is traded away as a bride to a "savage" in season one. Who selfishly treats her like a means to an end?

Evidence:
[S?E?] ===Across the Narrow Sea=== Reaching Astapor in Slaver's Bay with her fast-growing dragons, Daenerys Targaryen considers buying an army of the "Unsullied", renowned eunuch soldiers. A warlock masked as a young girl attempts to assassinate Daenerys with a scorpion-like creature, but is thwarted by Ser Barristan Selmy, Kingsguard to Daenerys' father, who swears his allegiance to her.
[S?E?] ====Season 1==== Viserys Targaryen is the exiled prince and heir of the Targaryen dynasty and is known as "The Beggar King" for his search for an army to recapture his throne. A narcissist, he is arrogant and self-centered, carin

...


🔍 PROMPT SENT TO OPENAI

You are a Game of Thrones expert.
Answer in one sentence, using ONLY the evidence lines below. If evidence is insufficient, say:
"I cannot find this in the provided evidence."
Include season/episode if present.

Question: "I shall have your head for this!"

Evidence:
[S1E10] # "I'll have your ugly head! # "You're nowhere near as murd'rous as... # "The lion in my bed" # King Robert lost his battle and... # He failed his final test # The lion ripped his balls off, and...
[S1E9] And my Lady Sansa... has begged mercy for her father. But they have the soft hearts of women. So long as I am your king... treason shall never go unpunished. Ser Ilyn... bring me his head!
[S1E9] treason shall never go unpunished. Ser Ilyn... bring me his head! No! Someone stop him! My son, this is madness.
[S3E4] Show us where he hides the food, or you'll get the same as him. Unhand her. I shall have your head for this. Quickly. Quickly. What's happening? I'm not going out there.
[S1E5] 

 99%|█████████▉| 164/165 [04:35<00:01,  1.36s/it]


🔍 PROMPT SENT TO OPENAI

You are a Game of Thrones expert.
Answer in one sentence, using ONLY the evidence lines below. If evidence is insufficient, say:
"I cannot find this in the provided evidence."
Include season/episode if present.

Question: "Gods help you Theon Greyjoy. Now you are truly lost."

Evidence:
[S2E6] You said no harm would come. Theon, please. Any last words, old man? Gods help you, Theon Greyjoy. Now you are truly lost. Theon, please don't. Do something! I'll do anything, please!
[S2E6] - Please, stop this. Please stop him. - Hush now, child. I'm off to see your father. You said no harm would come! You said no harm would come. Theon, please. Any last words, old man? Gods help you, Theon Greyjoy.
[S2E6] Now you are truly lost. Theon, please don't. Do something! I'll do anything, please! No, please stop it! Ghost, stay with us. Ghost!
[S4E6] I'm not tricking you, Theon. I'm saving you. Not Theon! Reek! - My name is Reek! - They're here. - lf they catch us in here, we'

100%|██████████| 165/165 [04:37<00:00,  1.68s/it]


📈 Evaluation Summary
Exact Match (EM):       0.000
F1 Score:               0.160
Semantic Similarity:    0.184
Saved detailed results → eval/funtrivia_agentic_eval.csv


In [47]:
!pip install langchain langchain-openai langchain-community sentence-transformers faiss-cpu rank-bm25

  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.1.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
  Using cached torch-2.9.0-cp313-none-macosx_11_0_arm64.whl.metadata (30 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
  Using cached safetensors-0.6.2-cp38-abi3-macosx_11_0_arm64.whl.metadata (4.1 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 2.2 MB/s  0:00:01 eta 0:00:01
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 3.0 MB/s  0:00:00 eta 0:00:01
Using cached marshmallow-3.26.1-py3-none-any.whl (50 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 3.7 MB/s 

In [55]:
# ==========================
# COSINE of Thrones — Multi-Agent Supervisor (Tool-Calling Pattern)
# ==========================
# Prereqs:
# pip install -U langchain langchain-core langchain-openai langchain-community faiss-cpu rank-bm25 tqdm

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.tools import tool
from langchain.agents import create_agent
from langchain_community.vectorstores import FAISS
from langchain_community.retrievers import BM25Retriever
from langchain_core.documents import Document
from tqdm import tqdm
import os, pandas as pd

# --- assume df_chunks or df_aug already loaded ---
def make_documents(df):
    docs = []
    for _, r in df.iterrows():
        docs.append(
            Document(
                page_content=str(r["text"]),
                metadata={
                    "season": int(r["season"]) if pd.notna(r["season"]) else None,
                    "episode": int(r["episode"]) if pd.notna(r["episode"]) else None
                }
            )
        )
    return docs

docs = make_documents(df_aug if "df_aug" in locals() else df_chunks)

# ==========================
# Safe FAISS Batch Embedding (≤300K tokens)
# ==========================
from tqdm import tqdm
import numpy as np

emb = OpenAIEmbeddings(model="text-embedding-3-large", api_key=os.getenv("OPENAI_API_KEY"))

faiss_store = None
batch_size = 50           # smaller = safer
max_chars = 8000          # truncate long chunks

for i in tqdm(range(0, len(docs), batch_size), desc="Embedding batches"):
    batch_docs = docs[i:i+batch_size]
    batch_texts = []
    batch_meta = []

    for d in batch_docs:
        text = d.page_content[:max_chars]
        if len(d.page_content) > max_chars:
            text += " [TRUNCATED]"
        batch_texts.append(text)
        batch_meta.append(d.metadata)

    try:
        if faiss_store is None:
            faiss_store = FAISS.from_texts(batch_texts, emb, metadatas=batch_meta)
        else:
            faiss_store.add_texts(batch_texts, metadatas=batch_meta)
    except Exception as e:
        print(f"⚠️  Batch {i//batch_size} failed ({len(batch_texts)} texts): {e}")
        # retry one by one if necessary
        for txt, meta in zip(batch_texts, batch_meta):
            try:
                faiss_store.add_texts([txt], metadatas=[meta])
            except Exception as e2:
                print("   ❌ Skipped one text due to embedding error:", str(e2)[:100])

print(f"✅ FAISS index built — {len(docs)} documents embedded safely.")

# --- build hybrid retriever ---
dense_retriever = faiss_store.as_retriever(search_kwargs={"k": 6})
bm25 = BM25Retriever.from_documents(docs)
bm25.k = 6

def hybrid_retrieve(query, topk=6):
    dense = dense_retriever.get_relevant_documents(query)
    lexical = bm25.get_relevant_documents(query)
    # merge & dedup by text content
    merged = {d.page_content: d for d in dense + lexical}
    return list(merged.values())[:topk]

# --- subagent tools ---
@tool("retrieve_context", description="Retrieve Game of Thrones evidence lines for a given question.")
def retrieve_tool(query: str):
    docs = hybrid_retrieve(query)
    txt = "\n".join(f"[S{d.metadata.get('season')}E{d.metadata.get('episode')}] {d.page_content}" for d in docs)
    return txt or "(no evidence found)"

@tool("summarize_context", description="Summarize or answer using provided Game of Thrones evidence.")
def summarize_tool(context: str):
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1, api_key=os.getenv("OPENAI_API_KEY"))
    prompt = (
        "You are a Game of Thrones expert.\n"
        "Use only the evidence below to answer concisely.\n\n"
        f"{context}"
    )
    res = llm.invoke(prompt)
    return res.content.strip()

# --- create supervisor agent ---
supervisor = create_agent(
    model="gpt-4o-mini",
    tools=[retrieve_tool, summarize_tool],
    system_prompt=(
        "You are the COSINE-of-Thrones supervisor agent. "
        "When asked a question, first call `retrieve_context` to fetch evidence, "
        "then call `summarize_context` with that evidence to produce the final answer."
    ),
)

# --- run one sample ---
query = "Who kills Tywin Lannister?"
result = supervisor.invoke({
    "messages": [{"role": "user", "content": query}]
})

print("\n🎬 Question:", query)
print("\n🧠 Supervisor Response:\n", result["messages"][-1].content)

Embedding batches: 100%|██████████| 349/349 [03:59<00:00,  1.45it/s]


✅ FAISS index built — 17414 documents embedded safely.


AttributeError: 'VectorStoreRetriever' object has no attribute 'get_relevant_documents'

In [2]:
from langchain_community.tools import DuckDuckGoSearchRun  # Initialize the tool
search_ddg_tool = DuckDuckGoSearchRun()

tools = [search_ddg_tool]

# Use latest langchain-compatible imports and avoid hub
try:
    from langchain.chat_models import ChatOpenAI
except Exception:
    from langchain_openai import ChatOpenAI

from langgraph.prebuilt import create_react_agent

# Build a compatible prompt (avoid deprecated hub API)
try:
    from langchain.prompts import ChatPromptTemplate
except Exception:
    from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an executor agent. Follow instructions and call tools when needed."),
        ("user", "{messages}"),
    ]
)
if hasattr(prompt, "pretty_print"):
    try:
        prompt.pretty_print()
    except Exception:
        pass

# Choose the LLM that will drive the agent (try both parameter names)
try:
    llm = ChatOpenAI(model_name="gpt-4o-mini")
except TypeError:
    llm = ChatOpenAI(model="gpt-4o-mini")

# create_react_agent API changed across versions. Try multiple call signatures and fallbacks.
from inspect import signature

agent_executor = None
errors = []

# Try calling with obvious kwarg names if the signature supports them
try:
    sig = signature(create_react_agent)
    params = sig.parameters.keys()
except Exception:
    sig = None
    params = ()

for kw in ("state_modifier", "prompt", "state_mod", "state_prompt"):
    if kw in params:
        try:
            agent_executor = create_react_agent(llm, tools, **{kw: prompt})
            print(f"create_react_agent called with {kw}=ChatPromptTemplate")
            break
        except Exception as e:
            errors.append((kw, str(e)))

# Try a plain-string prompt if templated prompt fails or isn't accepted
if agent_executor is None:
    try:
        fallback_prompt = "You are an executor agent. Follow instructions and call tools when needed.\n{messages}"
        # prefer 'state_modifier' if present, else try with any kw that didn't error
        if "state_modifier" in params:
            agent_executor = create_react_agent(llm, tools, state_modifier=fallback_prompt)
            print("create_react_agent called with state_modifier=string")
        elif "prompt" in params:
            agent_executor = create_react_agent(llm, tools, prompt=fallback_prompt)
            print("create_react_agent called with prompt=string")
        else:
            # last resort: call with positional args only
            agent_executor = create_react_agent(llm, tools)
            print("create_react_agent called with default signature (no prompt arg)")
    except Exception as e:
        errors.append(("fallback", str(e)))
        raise RuntimeError(
            "create_react_agent failed for all tested signatures. Errors: "
            + "; ".join(f"{k}: {v}" for k, v in errors)
        ) from e

# Basic sanity check
if not hasattr(agent_executor, "invoke"):
    raise RuntimeError("agent_executor created but does not expose 'invoke' method")

================================ System Message ================================

You are an executor agent. Follow instructions and call tools when needed.

================================ Human Message =================================

{messages}
create_react_agent called with prompt=ChatPromptTemplate


/var/folders/x8/rtqqwrlj1wb4l1fs1jvx5z_80000gn/T/ipykernel_90570/936097646.py:55: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent_executor = create_react_agent(llm, tools, **{kw: prompt})


In [5]:
response = agent_executor.invoke({"messages": [("user", "In games of thrones, Arthur Dayne was a member of the Kingsguard of which king?")]})
response['messages'][-1].content  # correct as of 11/19/24

"Ser Arthur Dayne was a member of the Kingsguard under King Aerys II Targaryen, also known as the Mad King. He served in this position during the time leading up to and including Robert's Rebellion, where he notably perished."

In [ ]:
# Let's use our existing vector store for retrieval with langgraph
from langchain_community.vectorstores import FAISS
faiss_store = FAISS.load_local("data/got_aug_faiss.bin", OpenAIEmbeddings(model="text-embedding-3-large", api_key=os.environ["OPENAI_API_KEY"]))
dense_retriever = faiss_store.as_retriever(search_kwargs={"k": 6})
bm25 = BM25Retriever.from_documents(make_documents(df_aug))
bm25.k = 6

# Make a tool that uses our hybrid retriever
@tool("retrieve_got_context", description="Retrieve Game of Thrones evidence lines for a given question.")
def retrieve_got_tool(query: str):
    docs = hybrid_retrieve(query)
    txt = "\n".join(f"[S{d.metadata.get('season')}E{d.metadata.get('episode')}] {d.page_content}" for d in docs)
    return txt or "(no evidence found)"

tools.append(retrieve_got_tool)
@tool("summarize_got_context", description="Summarize or answer using provided Game of Thrones evidence.")
def summarize_got_tool(context: str):
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1, api_key=os.getenv("OPENAI_API_KEY"))
    prompt = (

